Import modules

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(87)
torch.cuda.manual_seed(87)

Create classification data

In [ ]:
import numpy as np
from sklearn.datasets import make_classification

X, y = make_classification(
    n_samples=1000,
    n_features=20,
    n_informative=10,
    random_state=87
)
X, y = np.array(X, dtype=np.float32), np.array(y, dtype=np.int64)

print(X.shape, y.shape, y.mean())

Create classification model

In [ ]:
class ClassifierModule(nn.Module):
    def __init__(self, num_units=10, 
                 nonlin=F.relu, dropout=0.5):
        super(ClassifierModule, self).__init__()

        self.nonlin = nonlin
        self.dropout = nn.Dropout(dropout)
        self.dense0 = nn.Linear(20, num_units)
        self.dense1 = nn.Linear(num_units, 10)
        self.output = nn.Linear(10, 2)

    def forward(self, X):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        X = self.nonlin(self.dense1(X))
        X = F.softmax(self.output(X), dim=-1)
        return X

Defining and training neural net classifier

In [ ]:
from skorch import NeuralNetClassifier

net = NeuralNetClassifier(
    ClassifierModule,
    criterion=nn.CrossEntropyLoss,
    optimizer=torch.optim.SGD,
    max_epochs=20,
    lr=0.1,
    batch_size=32,
    iterator_train__shuffle=True,
    # device='mps'
)

net.fit(X, y)

Make predictions, classification

In [ ]:
y_pred = net.predict(X[:5])
y_proba = net.predict_proba(X[:5])
print(y_pred, y_proba, sep='\n')

Create regression data

In [ ]:
from sklearn.datasets import make_regression

X_regr, y_regr = make_regression(
    n_samples=1000,
    n_features=20,
    n_informative=10,
    random_state=87
)
X_regr, y_regr = (np.array(X_regr, dtype=np.float32),
                  np.array(y_regr, dtype=np.float32) / 100)
y_regr = y_regr.reshape(-1, 1)

print(X_regr.shape, y_regr.shape, y_regr.min(), y_regr.max())

Create regression model

In [ ]:
class RegressorModule(nn.Module):
    def __init__(self, num_units=10,
                 nonlin=F.relu, dropout=0.5):
        super(RegressorModule, self).__init__()

        self.nonlin = nonlin
        self.dropout = nn.Dropout(dropout)
        self.dense0 = nn.Linear(20, num_units)
        self.dense1 = nn.Linear(num_units, 10)
        self.output = nn.Linear(10, 1)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        X = self.nonlin(self.dense1(X))
        X = self.output(X)
        return X

In [ ]:
from skorch import NeuralNetRegressor

net_regr = NeuralNetRegressor(
    RegressorModule,
    criterion=nn.MSELoss,
    optimizer=torch.optim.SGD,
    max_epochs=20,
    lr=0.1,
    batch_size=32,
    iterator_train__shuffle=True,
    # device='mps'
)
net_regr.fit(X_regr, y_regr)

Make predictions, regression

In [ ]:
y_pred = net.predict(X_regr[:5])
print(y_pred)

Saving and loading the model

In [ ]:
import pickle, os

save_dir = "/Users/pepijnschouten/Desktop/Python_Scripts/" \
    "Python_scripts_Varia/Deep_Learning/Skorch/models"

file_name = "basic_usage_model.pkl"

# saving
with open(os.path.join(save_dir, file_name), "wb") as f:
    pickle.dump(net, f)
    print("Model saved successfully.")

# loading
with open(os.path.join(save_dir, file_name), "rb") as f:
    net = pickle.load(f)
    print("Model loaded successfully.")

Saving and laoding model parameters

In [ ]:
param_name = "basic_usage_model_params.pkl"

# saving
net.save_params(os.path.join(save_dir, param_name))

# initalizing
new_net = NeuralNetClassifier(  
    ClassifierModule,
    criterion=nn.CrossEntropyLoss,
    optimizer=torch.optim.SGD,
    max_epochs=20,
    lr=0.1,
    batch_size=32,
    iterator_train__shuffle=True,
    # device='mps'
).initialize()

# loading
new_net.load_params(os.path.join(save_dir, param_name))

Usage with sklearn Pipeline

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

net = NeuralNetClassifier(
    ClassifierModule,
    criterion=nn.CrossEntropyLoss,
    optimizer=torch.optim.SGD,
    max_epochs=20,
    lr=0.1,
    batch_size=32,
    iterator_train__shuffle=True,
    # device='mps'
)

pipe = Pipeline([
    ('scale', scaler),
    ('net', net)
])

train_history = pipe.fit(X, y)

y_proba = pipe.predict_proba(X)
y_pred = pipe.predict(X)

Using Callbacks

In [ ]:
from skorch.callbacks import EpochScoring

"""
    Passing a string, None (own scoring), function
"""

auc = EpochScoring(scoring='roc_auc', lower_is_better=False)

net = NeuralNetClassifier(
    ClassifierModule,
    criterion=nn.CrossEntropyLoss,
    optimizer=torch.optim.SGD,
    max_epochs=20,
    lr=0.1,
    batch_size=32,
    iterator_train__shuffle=True,
    callbacks=[auc],
    # device='cuda'
)

net.fit(X, y)

Usage with sklearn GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV

# show network prefixes
print(', '.join(net.prefixes_))

net = NeuralNetClassifier(
    ClassifierModule,
    criterion=nn.CrossEntropyLoss,
    optimizer=torch.optim.SGD,
    max_epochs=20,
    lr=0.1,
    optimizer__momentum=0.9,
    batch_size=32,
    train_split=False,
    verbose=0,
    # device='mps'
)

param_grid = {
    'lr': [0.05, 0.1],
    'module__num_units': [10, 20],
    'module__dropout': [0., 0.5],
    'module__nonlin': [F.relu, F.tanh],
    'optimizer__momentum': [0., 0.9],
}

gs = GridSearchCV(net, param_grid=param_grid,
                  refit=False,
                  cv=3,
                  scoring='accuracy',
                  verbose=2)
gs.fit(X, y)

In [ ]:
from pprint import pprint

# print best score and params
print(gs.best_score_)
pprint(gs.best_params_)